# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github.com/24p11/recode-with-mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [1]:
%cd /content/
!git clone https://github.com/24p11/recode-with-mistral-finetune.git

/content
Cloning into 'recode-with-mistral-finetune'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (500/500), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 500 (delta 266), reused 493 (delta 259), pack-reused 0 (from 0)
Receiving objects: 100% (500/500), 1016.34 KiB | 7.31 MiB/s, done.
Resolving deltas: 100% (266/266), done.


Install all required dependencies:

In [3]:
!pip install -r /content/recode-with-mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.0 MB/s eta 0:00:00

## Model download

In [4]:
!pip install huggingface_hub

In [5]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

In [ ]:
# Alternatively, you can download the model from mistral

# !wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2024-05-24 18:50:25--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.6.117, 104.26.7.117, 172.67.70.68, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.6.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  40.5MB/s    in 6m 3s   

2024-05-24 18:56:29 (38.1 MB/s) - ‘mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [ ]:
# !DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [ ]:
!ls /content/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [38]:
# navigate to the mistral-finetune directory
%cd /content/recode-with-mistral-finetune/example/

/content/recode-with-mistral-finetune/example


In [39]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/recode-with-mistral-finetune/sample_data/train_instruct.jsonl"  # Fill
  data: "/content/recode-with-mistral-finetune/sample_data/train_text.jsonl"  # Optionally fill with pretraining data
  eval_instruct_data: "/content/recode-with-mistral-finetune/sample_data/val_instruct.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/mistral7B_finetune_v1"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('mistral7B_finetune_v1.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)

In [40]:
# navigate to the mistral-finetune directory
%cd /content/recode-with-mistral-finetune/

/content/recode-with-mistral-finetune


In [41]:
! git pull

Already up to date.


In [42]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/mistral7B_finetune_v1.yaml

0it [00:00, ?it/s]Validating /content/recode-with-mistral-finetune/sample_data/train_text.jsonl ...

  0% 0/44 [00:00<?, ?it/s]
100% 44/44 [00:00<00:00, 390.84it/s]
1it [00:00,  8.80it/s]Validating /content/recode-with-mistral-finetune/sample_data/train_instruct.jsonl ...

  0% 0/44 [00:00<?, ?it/s]
100% 44/44 [00:00<00:00, 321.19it/s]
2it [00:00,  7.95it/s]
No errors! Data is correctly formatted!
Stats for /content/recode-with-mistral-finetune/sample_data/train_instruct.jsonl and /content/recode-with-mistral-finetune/sample_data/train_text.jsonl 
 -------------------- 
 {
    "expected": {
        "eta": "00:07:23",
        "data_tokens": 130976,
        "train_tokens": 6553600,
        "epochs": "50.04",
        "max_steps": 100,
        "data_tokens_per_dataset": {
            "/content/recode-with-mistral-finetune/sample_data/train_text.jsonl": "61704.0",
            "/content/recode-with-mistral-finetune/sample_data/train_instruct.jsonl": "69272.0"
        },
        "train_tokens

## Start training

In [45]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [46]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/mistral7B_finetune_v1
# ! rm -r /content/mistral7B_finetune_v1

In [47]:
# start training

!torchrun --nproc-per-node 1 -m train example/mistral7B_finetune_v1.yaml

2024-12-30 15:21:12.266811: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-30 15:21:12.283917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-30 15:21:12.304891: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-30 15:21:12.311255: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 15:21:12.326173: I tensorflow/core/platform/cpu_feature_guar

## Inference

In [48]:
!pip install mistral_inference

In [68]:
!pip install numba

In [70]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [78]:
cuda.select_device(0)
device = cuda.get_current_device()
device.reset()

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-66a34127-4b30-7bc5-0dcc-b8f445ef8962)


In [2]:
# prompt: libérer la mémoire gpu avec google collab

import gc
import torch

# Release GPU memory
gc.collect()
torch.cuda.empty_cache()

# Optionally, you can also try to delete large objects you no longer need
# Replace 'your_large_variable' with the actual name of your variable
# del your_large_variable
# gc.collect()
# torch.cuda.empty_cache()

# Check GPU memory usage again
!nvidia-smi

Mon Dec 30 16:17:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
from pathlib import Path
import torch
import datetime as dt
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import json

from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, SystemMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

finetune_model_name = "instruct_icd_v1"
source_model_path = "/content/mistral_models"
finetune_model_path = "/content/mistral7B_finetune_v1"
data_path = "/content/recode-with-mistral-finetune/sample_data/test_instruct.jsonl"

tokenizer = MistralTokenizer.from_file(source_model_path+"/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder(source_model_path)  # change to extracted model dir
model.load_lora(finetune_model_path+"/checkpoints/checkpoint_000100/consolidated/lora.safetensors")


In [6]:
def pred_codes_stat(data,tokenizer,model):
    system_message = SystemMessage(
        content="Vous êtes un modèle de langage en française spécialisé dans le codage des diagnostics selon la classification internationale des maladies version 10 (CIM-10) pour les résumés standardisés de sortie du programme de médicalisation des systèmes d'information français (PMSI). A partir des comptes rendus d'hospitalisation vous donnerez les codes diagnostics CIM-10 que l'on peut retenir pour le séjours en distiguant diagnostic principal, diagnostic relié et diagnostics associés.")
    user_message = UserMessage(
      content="Générez les codes CIM et leurs définitions pour le résumé du séjour suivant : {TEXT}".format(TEXT = data["messages"][1]["content"]))
    completion_request = ChatCompletionRequest(messages=[system_message , user_message])

    tokens = tokenizer.encode_chat_completion(completion_request).tokens

    out_tokens, _ = generate([tokens], model, max_tokens=300, temperature=0.7, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)

    result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
    regex_parenthess = r"\((.*?)\)"
    regex_codes = r"^[a-zA-Z]\d+"
    codes_pred = re.findall(regex_parenthess, result)
    codes_pred = [code for code in codes_pred if re.match(regex_codes, code)]
    codes_crh = re.findall(regex_parenthess, data["messages"][2]["content"])
    codes_crh = [code for code in codes_crh if re.match(regex_codes, code)]
    vp = [ code for code in codes_pred if code in codes_crh]
    fp = [ code for code in codes_pred if code not in codes_crh]
    fn = [ code for code in codes_crh if code not in codes_pred]

    result_tmp = pd.DataFrame([{
       "text":data["messages"][1]["content"],
       "text_codes_crh": data["messages"][2]["content"],
       "codes_crh":codes_crh,
       "text_codes_crh": result,
       "prediction" :codes_pred,
       "n_codes" : len(codes_crh),
       "n_pred" : len(codes_pred),
       "vp": vp,
       "n_vp": len(vp),
       "fp": fp,
       "n_fp": len(fp),
       "fn": fn,
       "n_fn": len(fn)}])
    return result_tmp


In [11]:
i = 0
limit = 1000000
result = pd.DataFrame()
print(dt.datetime.today().strftime("%Y-%m-%d %H:%M:%S")  +" - Begin prediction loop")
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    for idx, line in tqdm(enumerate(lines), total=len(lines)):

        data = json.loads(line)
        result_tmp = pred_codes_stat(data,tokenizer,model)
        result = pd.concat([result, result_tmp])

        if i>limit:
            break
        i+=1

print(dt.datetime.today().strftime("%Y-%m-%d %H:%M:%S")  +" - End prediction loop")
timeStamp = dt.datetime.today().strftime("_%Y-%m-%d-%H%M%S")
file_name = "test_results"+timeStamp+".csv"
result.to_csv("/content/recode-with-mistral-finetune/sample_data/"+file_name)
print("File "+file_name  + " saved")



2024-12-30 16:28:19 - Begin prediction loop


100%|██████████| 7/7 [00:49<00:00,  7.10s/it]

2024-12-30 16:29:09 - End prediction loop
File test_results_2024-12-30-162909.csv saved


In [12]:
precision = np.sum(result["n_vp"]) / np.sum(result["n_codes"])
recall = np.sum(result["n_vp"]) / np.sum(result["n_pred"])
f1_score = 2 * precision * recall / (precision + recall)
acc = result[(result.n_fp ==0) & (result.n_fn==0)].shape[0] / result.shape[0]
print("Precision = "  + str(precision))
print("Recall = "  + str(recall))
print("F1-Score = "  + str(f1_score))
print("Accuracy = "  + str(acc))

Precision = 0.7333333333333333
Recall = 0.7096774193548387
F1-Score = 0.7213114754098361
Accuracy = 0.0


In [8]:
print(dt.datetime.today().strftime("%Y-%m-%d %H:%M:%S")  +" - End prediction loop")
timeStamp = dt.datetime.today().strftime("_%Y-%m-%d-%H%M%S")
result.to_csv("/content/recode-with-mistral-finetune/sample_data/test_results"+timeStamp+".csv")
print("File test_results"+timeStamp + " saved")

2024-12-30 16:23:37 - End prediction loop
File test_results_2024-12-30-162337 saved


In [9]:
result

,text,text_codes_crh,codes_crh,prediction,n_codes,n_pred,vp,n_vp,fp,n_fp,fn,n_fn
0,Générez le codage CIM-10 du résumé strandisé d...,Codes CIM 10 retenus pour le résumé strandisé ...,"[R102, E1198, E8768, I10, C795]","[R410, E8768, E1198, I10, C795]",5,5,"[E8768, E1198, I10, C795]",4,[R410],1,[R102],1
0,Générez le codage CIM-10 du résumé strandisé d...,Codes CIM 10 retenus pour le résumé strandisé ...,"[C798, E559, N185, E8768]","[C795, E559, N185, E8768]",4,4,"[E559, N185, E8768]",3,[C795],1,[C798],1
0,Générez le codage CIM-10 du résumé strandisé d...,Codes CIM 10 retenus pour le résumé strandisé ...,"[Z511, C189+0]","[Z511, C795, D124]",2,3,[Z511],1,"[C795, D124]",2,[C189+0],1
0,Générez le codage CIM-10 du résumé strandisé d...,Codes CIM 10 retenus pour le résumé strandisé ...,"[Z511, C254, I255, E8718, R33, E1190, I10, N185]","[Z511, C797, E1190, N185, E8718, R33, I255]",8,7,"[Z511, E1190, N185, E8718, R33, I255]",6,[C797],1,"[C254, I10]",2
0,Générez le codage CIM-10 du résumé strandisé d...,Codes CIM 10 retenus pour le résumé strandisé ...,"[Z511, C187, R33]","[Z511, C187, R33]",3,3,"[Z511, C187, R33]",3,[],0,[],0
0,Générez le codage CIM-10 du résumé strandisé d...,Codes CIM 10 retenus pour le résumé strandisé ...,"[Z511, C793, C787, G473]","[Z511, C798, C795, C787, G473]",4,5,"[Z511, C787, G473]",3,"[C798, C795]",2,[C793],1
0,Générez le codage CIM-10 du résumé strandisé d...,Codes CIM 10 retenus pour le résumé strandisé ...,"[R101, C787, N185, R33]","[R101, C787, R33, N185]",4,4,"[R101, C787, R33, N185]",4,[],0,[],0
